In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

In [9]:
df = pd.read_csv("D:\Download\LP-IV-datasets (1)\LP-IV-datasets\ECGdataset(Ass4)\ecg_autoencoder_dataset.csv",header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,131,132,133,134,135,136,137,138,139,140
0,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,-1.818286,-1.250522,-0.477492,...,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137,1
1,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250,1
2,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490659,-1.183580,-0.394229,...,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797,1
3,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564,1
4,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456,1


In [18]:
x = df.drop([140],axis = 1).values
y = df[140].values

In [19]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

In [23]:
minmax = MinMaxScaler(feature_range=(0,1))
x_train = minmax.fit_transform(x_train)
x_test = minmax.fit_transform(x_test)

In [33]:
encoder = Sequential(Dense(128,activation='relu',input_shape=(x_train.shape[1],)))
decoder = Sequential(Dense(x_train.shape[1],activation='softmax'))

In [34]:
autoencoder = Sequential([encoder,decoder])

In [40]:
autoencoder.compile(optimizer='sgd',loss='mae',metrics='mae')

In [41]:
autoencoder.fit(x_train,x_train,epochs=5,batch_size=32,validation_data=(x_test,x_test))

Epoch 1/5
110/110 [==============================] - 2s 11ms/step - loss: 0.5886 - mae: 0.5886 - val_loss: 0.5888 - val_mae: 0.5888
Epoch 2/5
110/110 [==============================] - 1s 6ms/step - loss: 0.5886 - mae: 0.5886 - val_loss: 0.5888 - val_mae: 0.5888
Epoch 3/5
110/110 [==============================] - 1s 7ms/step - loss: 0.5886 - mae: 0.5886 - val_loss: 0.5888 - val_mae: 0.5888
Epoch 4/5
110/110 [==============================] - 1s 6ms/step - loss: 0.5886 - mae: 0.5886 - val_loss: 0.5888 - val_mae: 0.5888
Epoch 5/5
110/110 [==============================] - 1s 7ms/step - loss: 0.5886 - mae: 0.5886 - val_loss: 0.5888 - val_mae: 0.5888


In [37]:
loss = autoencoder.evaluate(x_test,x_test)
print(loss)

47/47 [==============================] - 0s 4ms/step - loss: 0.5888 - mae: 0.5888
[0.5888322591781616, 0.5888322591781616]


In [49]:
out_data = autoencoder.predict(x_test)
mse = np.mean(np.power(out_data - x_test,2),axis=1)
threshold = np.percentile(mse,95)
outlie= mse>threshold

print(confusion_matrix(y_test,outlie))
print(classification_report(y_test,outlie))

47/47 [==============================] - 0s 3ms/step
[[571  53]
 [854  22]]
              precision    recall  f1-score   support

           0       0.40      0.92      0.56       624
           1       0.29      0.03      0.05       876

    accuracy                           0.40      1500
   macro avg       0.35      0.47      0.30      1500
weighted avg       0.34      0.40      0.26      1500



In [52]:
no_outli = np.sum(outlie)
print(no_outli)
no_ano = np.sum(y_test[outlie]==1)
print(no_ano)

75
22
